In [ ]:
from time import sleep
from tqdm.notebook import tqdm
from multiprocessing import Pool, freeze_support

def progresser(n):
    # This line is the strange hack
    print(' ', end='', flush=True)

    text = "progresser #{}".format(n)
    for i in tqdm(range(5000), desc=text, position=n):
         sleep(0.001)
        
if __name__ == '__main__':
    freeze_support()
    L = list(range(10))
    print()
    Pool(2).map(progresser, L)


In [ ]:
import time
from tqdm.notebook import tqdm
for i in tqdm(range(10)):
    time.sleep(0.1)


  0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
from jsonColumnParser import *
import multiprocessing

from typing import Any, Callable
from config import load_config
import pathlib
# import json
import orjson as json  # 能加速5%左右
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from tqdm import tqdm
from datetime import datetime


In [ ]:
multiprocessing.freeze_support()
multiprocessing.set_start_method('spawn')  # GPT说可以增加跨平台稳定性，没有测过
config = load_config()
num_file = 8
input_dir = pathlib.Path(r'E:\data_mining\dm-homework\split_file\10G_data_new')
output_dir = pathlib.Path("processed_data") / input_dir.name
output_dir.mkdir(parents=True, exist_ok=True)
args = [
    (
        input_dir / f"part.{i}.parquet",
        output_dir / f"part.{i}.parquet",
        pa.schema([
            pa.field('avg_price', pa.float64()),
            pa.field('categories', pa.string()),
            pa.field('items', pa.binary()),
            pa.field('item_count', pa.int64()),
            pa.field('payment_method', pa.string()),
            pa.field('payment_status', pa.string()),
            pa.field('purchase_date', pa.float64()),
        ]),
        "purchase_history",
        parse_purchase_history,
        1024,
        i,
    ) for i in range(num_file)
]
with multiprocessing.Pool() as pool:
    pool.starmap(parse_json_column, args)
tqdm.write("所有任务完成")